# Database Widget
- Allows user to add and remove databases and graphs
- Updates in "real time" (polling) to reflect the current status
- Shows database and graph metadata

In [ ]:
from katana.remote import Client
c = Client(address="host.docker.internal:8080")

c.widgets().databases()

# Opererations Widget
- Displays a list of operations sorted by type
- Allows user to filter by several criteria
- Allows cancellation of operations

In [ ]:
from katana.remote import Client
c = Client(address="host.docker.internal:8080")

c.widgets().operations()

# Web Dashboard
- All widgets on one webpage in a new tab
- Allows users to monitor status and make changes in real time independent of notebook progress
- Prevents "blocking" of widget display by previous cell execution

In [ ]:
from katana.remote import Client
c = Client(address="host.docker.internal:8080")

c.widgets().dashboard()

### Test 1: CSV Import and Query
- Note the change in operations widget showing the import
- When import is finished, note the operations widget showing the cypher query
- (optional): Use the operations widget to cancel the import before completion

In [ ]:
import os                                                                        
import time                                                                      
import json

from katana import remote
from katana.remote import import_data
from katana_enterprise import async_to_sync

os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

num_partitions = 2
cluster_name = "katana-internal16"

client = remote.Client()
async_client = remote.aio.Client()

dbs = client.databases()
for db in dbs:
    print(db.database_id, db.name)

graph1 = client.create_graph(num_partitions=num_partitions)

print(f"Importing graph from CSV files into graph {graph1} ...")
import_data.csv(
    graph1,
    input_node_path="gs://katana-demo-datasets/pp/mapping/node_list_2M_full_gcp.txt",
    input_edge_path="gs://katana-demo-datasets/pp/mapping/edge_list_2M_full_gcp.txt",
    input_dir="gs://katana-demo-datasets/pp/csv",
    #have_headers=True,
    files_have_headers=True,
    data_delimiter="|",
    schema_delimiter="|",
    partition_policy="random-oec",
)

query = "MATCH (a) RETURN count(a);"
result = graph1.query(query)
print(result[0])

### Test 2: Import into newly greated database and graph
- From the Database widget, create a new database
- Select the new database, and create a new graph
- Select the new graph, and copy paste the database ID and graph ID into the cell below
- Note the CSV import into the newly created graph
- Note the query completes on the new imported data
- (optional): Use the operations widget to cancel the import before completion

In [ ]:
import os   
from katana import remote
from katana.remote import import_data, Graph

os.environ["KATANA_SERVER_ADDRESS"] = "host.docker.internal:8080"

client = remote.Client()

# Copy and paste graph id and database id from Database widget here.
# In the future, clicking the "Copy" button on the graph menu will grab the graph and database IDs
graph_id="huY4uZj2huCpJDz2XUyCitTqwXb2BXbGsdTexcC4YAL"
database_id="2pteSaQLkvssMbMF4ikTgBvY6zfKoPdWC4t9xHNJApC7"

graph2 = Graph(client, graph_id=graph_id, database_id=database_id, version="0")

import_data.csv(
    graph2,
    input_node_path="gs://katana-demo-datasets/pp/mapping/node_list_2M_full_gcp.txt",
    input_edge_path="gs://katana-demo-datasets/pp/mapping/edge_list_2M_full_gcp.txt",
    input_dir="gs://katana-demo-datasets/pp/csv",
    #have_headers=True,
    data_delimiter="|",
    schema_delimiter="|",
    partition_policy="random-oec",
    #num_partitions=num_partitions,
)

query = "MATCH (a) RETURN count(a);"
result = graph1.query(query)
print(result[0])